In [ ]:
# 本程式碼修改自陳薈筑和柯俊杰之兩版速度反演程式碼，以作為112-2地震學原理上課使用。
# 修改目的: 融合兩版程式碼，建立class和function，以提高程式碼的可讀性與可擴展性。
# 修改目的: inversion區域挖空，提供註解給學生自行完成。
# This code is modified from the velocity inversion codes by Hui-Chu Chen and Jun-jie Ke, for use in the Principles of Seismology (112-2) course.
# Purpose of modification: Integrate two versions of the code, establish classes and functions to improve code improve code readability and scalability.
# Purpose of modification: Leave the inversion section blank, provide comments for students to complete independently.

In [1]:
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
class Love_wave():
    # parameter in SI unit
    def __init__(self, n, structure):
        
        # shallow structure
        self.H  = structure[0]                           # thickness(m) 
        self.b1 = structure[1]                           # velocity (m/s)
        self.d1 = structure[2]                           # density  (kg*m^-3)
        self.u1 = self.b1**2*self.d1                     # shear modulus
        
        # deep background
        self.b2 = structure[3]                           # velocity (m/s)
        self.d2 = structure[4]                           # density  (kg*m^-3)
        self.u2 = self.b2**2*self.d2                     # shear modulus
        
        # model parameter
        self.n    = n                                    # nth overtone
        self.intv = 1/(10+abs(0.3-n)**2)                 # calculating step
        self.num  = int((self.b2-self.b1)/self.intv+1)   # calculating times
        
        # modeling
        self.c = np.linspace(self.b1, self.b2, self.num) # phase velocity
    
    def overtone(self, w, plot=True):
        dv1 = np.sqrt(self.b1**(-2) - self.c**(-2))
        dv1[dv1==0.0] = 10**-16
        dv2 = np.sqrt(self.c**(-2)  - self.b2**(-2))
        w_y = dv1*self.H*w
        RHS = (dv2*self.u2)/(dv1*self.u1)
        LHS = np.tan(w_y)
        
        # numerical solution
        fi = []
        x  = []
        y  = []
        for i in range(1,self.c.shape[0]):
            aa = (RHS[i-1]-LHS[i-1])*(RHS[i]-LHS[i])
            if aa <= 0:
                fi.append(i)
        for i in range(len(fi)//2):
            j = np.where(w_y == w_y[fi[2*i]])[0][0]
            x.append(w_y[j])
            y.append(RHS[j])
            print(f'{i} overtune: {x[i].round(3)}, {y[i].round(3)}')
        if plot==True:            
            LHS[:-1][np.diff(LHS) < 0] = np.nan
            plt.plot(w_y, LHS, 'b', label='LHS')        
            plt.plot(w_y, RHS, 'r', label='RHS')
            plt.scatter(x, y, c='g')
            for i in range(len(x)):
                plt.text(x[i]+0.1, y[i]+0.1, f'n={i}')
            plt.hlines(0.0, min(w_y), max(w_y), 'k')
            plt.xlim(min(w_y), max(w_y))
            plt.ylim(-5.0, 40.0)
            plt.title(f'ω = {w}')
            plt.legend(loc='upper right')
            plt.show()
        
    def frequency(self):
        dv1 = np.sqrt(self.b1**(-2) - self.c**(-2))
        dv1[dv1==0.0] = 10**-16
        dv2 = np.sqrt(self.c**(-2)  - self.b2**(-2))
        y   = dv1*self.H
        # analytic solution
        w = np.arctan( (self.u2*dv2)/(self.u1*dv1) )
        w = (w + self.n*np.pi)/self.H/dv1
        return w, self.c

    def velocity(self, w): # U(c)
        k = w/self.c
        n = self.c.shape[0]
        dc_dk = np.zeros([n])
        for i in range(1, n):
            dc_dk[i] = (self.c[i]-self.c[i-1])/(k[i]-k[i-1])
        U = self.c + k*dc_dk
        U[0] = self.c[0]
        return U, self.c

In [3]:
def inversion(structure, C_ref, C_obs, omega, delta):
    m_ref = [structure[1], structure[3]]
    delV1 = delta[0] # perturbation of V1
    delV2 = delta[1] # perturbation of V2
    
    d = []
    for n in range(len(omega)):
        d.append(C_obs[n] - C_ref[n])
    d = np.array(d)
    G = np.zeros([4, 2])
    
    structure[1] += delV1
    perturbation1 = Love_wave(0, structure)
    w_m, c  = perturbation1.frequency()
    for n in range(len(omega)):
        w = omega[n]
        w_err = abs(w_m - w)
        i = np.where(w_err == min(w_err))[0][0]
        G[n, 0] = (c[i]-C_ref[n])/delV1
        
    structure[3] += delV2
    
    perturbation2 = Love_wave(0, structure)
    w_m, c  = perturbation2.frequency()
    #### inversion ####
    # create G & calculate m:
    # tip: delta_d = f'(m0)·delta_m -> d=G·m
    # tip: useful inversion tool by python package: np.linalg.inv()
    #### inversion ####
    return m

f = np.array([0.08,0.15,0.18,0.22])
omega = 2*np.pi*f
structure = [10000.0,    # 1 thickness(m) 
              3000.0,    # 1 velocity (m/s) 
              2800.0,    # 1 density  (kg*m^-3)
              5000.0,    # 2 velocity (m/s)
              3300.0]    # 2 density  (kg*m^-3)
C_obs = [4130,3529,3427,3346]

#### inversion ####
C_ref = [1,2,3,4] ## calculate the reference phase velocity for each frequency/omega
delta = [1, 1] ## try to give the perturbation of V1 and V2

#### inversion ####
m = inversion(structure, C_ref, C_obs, omega, delta)
print('\n>> Inversion result:')
print('β1 = ', m[0].round(0), 'm/s')
print('β2 = ', m[1].round(0), 'm/s')

NameError: name 'm' is not defined